<a href="https://colab.research.google.com/github/MinsooKwak/finetuning/blob/main/test/fine_tuning_GPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Fine Tuning

- 사전 학습 모델을 새로운 데이터셋에 맞춰 추가 학습을 수행하는 과정
- 현재 Fine-tuning 제공 모델 3가지
  - GPT3.5-turbo, Davinci-002, Babbage-002
- 데이터 형태를 맞춰주는 것이 일이기 때문에 많이 사용되지는 않음

- 과금 정보 : 학습에 대한 과금 추가

| Fine-tuning 제공 모델 | 학습 과금 (1K) | 입력 (1K) | 출력(1K) |
| --- | --- | --- | --- |
| GPT-3.5-turbo | $0.0080 | 0.0030 | 0.0060 |
| Davinci-002 | 0.0060 | 0.0120 | 0.0120 |
| Babbage-002 | 0.0004 | 0.0016 | 0.0016 |

- 모델에 따라 준비해야 할 데이터 형태가 상이함

  1. gpt-3.5-turbo
  ```
  [
    {"messages" : [
          {"role":"system",
            "content" : "Marv is a fatual chatbot that is also sarcastric"},
          {"role" : "user",
            "content" : "What's the captial of france?"},
          {"role" : "assistant",
            "content" : "Paris, as if everyone doesn't know that already"}
                  ]
                    },
    {"messages" : [
        {"role" : "system",
          "content" : "Marv is a factual chatbot that is also sarcastic"},
        {"role" : "user",
          "content" : "Who wrote Romeo and Juliet?"},
        {"role" : "assistant",
          "content" : Oh, jsut some guy names William Shapkespeare. Ever heard of him?"}
                  ]
                    }
  ]
  ```

  2. baggage-002, davinci-002
  ```
  [
    {"prompt" : <prompt_text1>, "completion" : <ideal generated text>},
    {"prompt" : <prompt_text2>, "completion" : <ideal gnerated text2>}
  ]
  ```


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# 초기 전처리한 데이터 import
adaptor_df = pd.read_csv("/content/drive/MyDrive/쇼퍼하우스/GenAilee/Data/sentiment_data/refined/adaptor_modified_score.csv")
pad_df = pd.read_csv("/content/drive/MyDrive/쇼퍼하우스/GenAilee/Data/sentiment_data/refined/pad_modified_score.csv")
wireless_df = pd.read_csv("/content/drive/MyDrive/쇼퍼하우스/GenAilee/Data/sentiment_data/refined/wireless_modified_score.csv")
maskpack_df = pd.read_csv("/content/drive/MyDrive/쇼퍼하우스/GenAilee/Data/sentiment_data/refined/maskpack_modified_score.csv")
leggings_df = pd.read_csv("/content/drive/MyDrive/쇼퍼하우스/GenAilee/Data/sentiment_data/refined/leggings_modified_score.csv")

In [3]:
adaptor_df.head(3)

,Unnamed: 0,brand_name,product_name,write_dt_lst,rating_lst,content_lst,len,repeated_final,review_type,tokenized_sentence,detail_review_type,morphs,Tokenized_morphs_list,match_dic,polarity,sentiment
0,3,스카이트립,여행 멀티 플러그 해외 여행용 어댑터 일본 멀티탭 콘센트 멀티 어댑터 유럽 중국,20240716,5,돈 추가해서 구매하는 게 더 좋네요 금액 차이 얼마 안 나는데 기능은 더 좋아요,44,0,general,돈 추가해서 구매하는 게 더 좋네요,service,"['추가', '더', '좋다']","추가, 더, 좋다",['좋다'],2,1
1,3,스카이트립,여행 멀티 플러그 해외 여행용 어댑터 일본 멀티탭 콘센트 멀티 어댑터 유럽 중국,20240716,5,돈 추가해서 구매하는 게 더 좋네요 금액 차이 얼마 안 나는데 기능은 더 좋아요,44,0,general,금액 차이 얼마 안 나는데 기능은 더 좋아요,service,"['나다', '기능', '더', '좋다']","나다, 기능, 더, 좋다",['좋다'],2,1
2,6,스카이트립,여행 멀티 플러그 해외 여행용 어댑터 일본 멀티탭 콘센트 멀티 어댑터 유럽 중국,20240714,5,콤팩트한 디자인과 실용성에 높은 점수 주었습니다,26,0,general,콤팩트한 디자인과 실용성에 높은 점수 주었습니다,product,"['콤팩트', '디자인', '실용', '높다', '주다']","콤팩트, 디자인, 실용, 높다, 주다",['높다'],1,1


In [4]:
adaptor_fine_df = adaptor_df[['tokenized_sentence','sentiment']]
adaptor_fine_df.head()

,tokenized_sentence,sentiment
0,돈 추가해서 구매하는 게 더 좋네요,1
1,금액 차이 얼마 안 나는데 기능은 더 좋아요,1
2,콤팩트한 디자인과 실용성에 높은 점수 주었습니다,1
3,배송이 너무 빠릅니다,1
4,좋은 제품 저렴하게 구입 잘했습니다,1


In [5]:
adaptor_fine_df.columns = ['prompt', 'completion']
adaptor_fine_df.head()

,prompt,completion
0,돈 추가해서 구매하는 게 더 좋네요,1
1,금액 차이 얼마 안 나는데 기능은 더 좋아요,1
2,콤팩트한 디자인과 실용성에 높은 점수 주었습니다,1
3,배송이 너무 빠릅니다,1
4,좋은 제품 저렴하게 구입 잘했습니다,1


In [6]:
adaptor_fine_df.completion = adaptor_fine_df.completion.map({0: '중립', 1: '긍정', -1: '부정'})
adaptor_fine_df.head()

<ipython-input-6-1f805e2b502b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  adaptor_fine_df.completion = adaptor_fine_df.completion.map({0: '중립', 1: '긍정', -1: '부정'})


,prompt,completion
0,돈 추가해서 구매하는 게 더 좋네요,긍정
1,금액 차이 얼마 안 나는데 기능은 더 좋아요,긍정
2,콤팩트한 디자인과 실용성에 높은 점수 주었습니다,긍정
3,배송이 너무 빠릅니다,긍정
4,좋은 제품 저렴하게 구입 잘했습니다,긍정


jsonl로 만들기

In [7]:
adaptor_fine_df.to_json('adaptor_fine.jsonl', orient='records', lines=True, force_ascii=False)

In [8]:
adaptor_json_test = pd.read_json('adaptor_fine.jsonl', lines=True)
adaptor_json_test.head(3)

,prompt,completion
0,돈 추가해서 구매하는 게 더 좋네요,긍정
1,금액 차이 얼마 안 나는데 기능은 더 좋아요,긍정
2,콤팩트한 디자인과 실용성에 높은 점수 주었습니다,긍정


In [9]:
adaptor_json_test.completion.value_counts().reset_index()

,completion,count
0,긍정,7167
1,중립,3243
2,부정,628


- 라벨링된 데이터가 완벽하진 않지만, 파인튜닝한다는 것에 우선 초점

In [10]:
# JSONL 파일을 직접 읽고 출력하기
with open('adaptor_fine.jsonl', 'r', encoding='utf-8') as file:
    for line in file:
        print(line.strip())  # 각 줄을 출력

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
{"prompt":"무엇보다 해외도 되는데 멀티어댑터까지 되니 좋은 것 같습니다","completion":"긍정"}
{"prompt":"감사히 잘 사용하겠습니다","completion":"긍정"}
{"prompt":"업무상 미주와 호주 쪽으로 이동이 잦습니다","completion":"중립"}
{"prompt":"동료가 쓰는 것 보고 구매했고 레이오버 때 요긴하게 썼습니다","completion":"중립"}
{"prompt":"불량은 없는 거 같고 공식이라 믿고 구매 미리 사용자로써 적극 추천드립니다","completion":"중립"}
{"prompt":"출국이 얼마 남지 않은 상태에서 급하게 주문했는데 정말 정말 다행히도 바로 다음날 받았어요","completion":"중립"}
{"prompt":"궁금한 점이 많아 구매 전 문의를 드렸었는데 친절하고 자세히 설명해 주셔서 믿고 구매할 수 있었습니다ㅇ","completion":"긍정"}
{"prompt":"9 여행 잘 다녀오겠습니다","completion":"중립"}
{"prompt":"감사해요","completion":"긍정"}
{"prompt":"2번째 구매입니다","completion":"중립"}
{"prompt":"국내 배송이 생겨서 이번에는 빠르게 받았네요","completion":"긍정"}
{"prompt":"역시 국내 배송이 최고예요","completion":"긍정"}
{"prompt":"해외 직구 특성상 배송이 느렸지만 판매자분이 친절하게 응대해 주시고 제품도 꼼꼼히 포장돼서 와서 만족합니다","completion":"긍정"}
{"prompt":"곧 호주로 어학연수 갈 예정이라 구입했는데 유용하게 잘 사용하겠습니다","completion":"긍정"}
{"prompt":"배송은 2주 정도 소요됩니다","completion":"중립"}
{"prompt":"급하신 경우는 자제 제품은 좋네요","completion":"긍정"}
{"promp

In [ ]:
#!pip install openai

In [11]:
# config 정보 (api key)
import os
from openai import OpenAI
import config

client = OpenAI(
	api_key = config.OPEN_AI_API_KEY
	)

In [12]:
# fine-tune 목적 파일 업로드
upload_response = client.files.create(
	file = open('adaptor_fine.jsonl', "rb"),
	purpose = 'fine-tune'
	)

In [13]:
file_id = upload_response.id
file_id

'file-A2GShhwFiJZFwrbnbSjjbKSC'

In [14]:
# fine-tuning 모델 생성
fine_tune_response = client.fine_tuning.jobs.create(
	training_file = file_id,  # job id 넣어주면 됨 (file-A2GShhwFiJZFwrbnbSjjbKSC)
	model = "davinci-002"
)

fine_tune_response

FineTuningJob(id='ftjob-JOKI8rtP8FwnDQzIJbFAaco7', created_at=1725636039, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='davinci-002', object='fine_tuning.job', organization_id='org-eJfKnt3mOtGBdEFIoNbtRarB', result_files=[], seed=1262703664, status='validating_files', trained_tokens=None, training_file='file-A2GShhwFiJZFwrbnbSjjbKSC', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None)

In [15]:
# fine-tuning job 리스트 10개 나열
client.fine_tuning.jobs.list(limit=10)

SyncCursorPage[FineTuningJob](data=[FineTuningJob(id='ftjob-JOKI8rtP8FwnDQzIJbFAaco7', created_at=1725636039, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='davinci-002', object='fine_tuning.job', organization_id='org-eJfKnt3mOtGBdEFIoNbtRarB', result_files=[], seed=1262703664, status='validating_files', trained_tokens=None, training_file='file-A2GShhwFiJZFwrbnbSjjbKSC', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None), FineTuningJob(id='ftjob-Tq7ES5MCX7JObfzTYoYK7LS7', created_at=1725615618, error=Error(code='invalid_training_file', message='The job failed due to an invalid training file. Expected file to have JSONL format with prompt/completion keys with string values. `completion` value on line 1 is not a string.', param='training_file'), fine_tuned_model=None, finished_at=None, hyperparameter

In [17]:
# fine-tuning 상태 확인
client.fine_tuning.jobs.retrieve("ftjob-Tq7ES5MCX7JObfzTYoYK7LS7")  # job id

FineTuningJob(id='ftjob-Tq7ES5MCX7JObfzTYoYK7LS7', created_at=1725615618, error=Error(code='invalid_training_file', message='The job failed due to an invalid training file. Expected file to have JSONL format with prompt/completion keys with string values. `completion` value on line 1 is not a string.', param='training_file'), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='davinci-002', object='fine_tuning.job', organization_id='org-eJfKnt3mOtGBdEFIoNbtRarB', result_files=[], seed=1440086534, status='failed', trained_tokens=None, training_file='file-ZK4vxAhfQAFr2g5Ym3IbXgSv', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None)

In [18]:
# fine-tuning job에서 10개 이벤트 나열
fine_tune_events = client.fine_tuning.jobs.list_events(fine_tuning_job_id = fine_tune_response.id)
fine_tune_events

SyncCursorPage[FineTuningJobEvent](data=[FineTuningJobEvent(id='ftevent-A7ljsNnNVVqY7xPszmTTrWkj', created_at=1725636064, level='info', message='Fine-tuning job started', object='fine_tuning.job.event', data=None, type='message'), FineTuningJobEvent(id='ftevent-PR0BEb56uXT2gDw3RFi3Uzb5', created_at=1725636063, level='info', message='Files validated, moving job to queued state', object='fine_tuning.job.event', data={}, type='message'), FineTuningJobEvent(id='ftevent-570ZiZhp80oMBxPvTRytSpcy', created_at=1725636040, level='info', message='Validating training file: file-A2GShhwFiJZFwrbnbSjjbKSC', object='fine_tuning.job.event', data={}, type='message'), FineTuningJobEvent(id='ftevent-iILbIrnBsX0NzFopmUpP1bVH', created_at=1725636039, level='info', message='Created fine-tuning job: ftjob-JOKI8rtP8FwnDQzIJbFAaco7', object='fine_tuning.job.event', data={}, type='message')], object='list', has_more=False)

- https://platform.openai.com/finetune 여기서 모델 확인

In [23]:
pad_df_sample = pad_df[['tokenized_sentence','sentiment']]
pad_df_sample.head()

,tokenized_sentence,sentiment
0,강아지 2마리와 살다 보니 배변패드가 많이 필요하네요,0
1,배변 흔적은 바로바로 치우기 때문에 절약형 패드가 정말 많이 도움 됩니다,1
2,항상 쓰던 거예요,0
3,가성비 좋고 배송도 빠르고,1
4,짱,1


In [25]:
pad_df_sample[pad_df_sample['sentiment']==-1].head()

,tokenized_sentence,sentiment
36,패드 낭비가 너무 심해서 마지막으로 산 거고 이제 안 써보려고요,-1
186,기존보다 도톱하니까 배변 실수가 줄더라고요,-1
251,이것만 한 대형견 패드는 없는 거 같아요,-1
265,사이즈가 커서 아기가 실수를 안 해서 더 맘에 들어요,-1
326,35킬로 몰티즈 남아인데 가끔 패드 끝 쪽에서 쉬하면 선 안쪽에서 다 흡수가 안되고...,-1


In [26]:
# fine-tuning한 모델 사용하기
completion1 = client.completions.create(
	model = "ft:davinci-002:personal::A4VGo4xG",
	prompt = "배변 흔적은 바로바로 치우기 때문에 절약형 패드가 정말 많이 도움 됩니다.")

print(completion1.choices[0].text)

중립긍정긍정정정긍정정


In [27]:
completion2 = client.completions.create(
	model = "ft:davinci-002:personal::A4VGo4xG",
	prompt = "패드 낭비가 너무 심해서 마지막으로 산 거고 이제 안 써보려고요	")

print(completion2.choices[0].text)

부정정부정부부정부부정부부정부부


In [28]:
completion3 = client.completions.create(
	model = "ft:davinci-002:personal::A4VGo4xG",
	prompt = "항상 쓰던 거예요.")

print(completion3.choices[0].text)

중립중립중립중립
